# Automating Azure AD B2C creation with PowerShell Core
[Link to Article](https://codez.deedx.cz/posts/automating-azure-ad-b2c-creation-with-powershell/)

### B2C tenant creation and provisioning
- [Full script can be found on GitHub](https://github.com/msimecek/AAD-automations/blob/main/b2c/Create-AzureB2C.ps1), this post walks through key parts of the solution and highlights some of the complications that were solved.

The process is designed as a series of PowerShell functions which could work individually or as a chain. These are the steps:

1. Create B2C tenant
2. Sign in to Microsoft Graph PowerShell
3. Create custom attribute
4. Create user flows
5. Create app registrations
6. Assign Microsoft Graph access
7. Grant admin consent
8. Import users

### Open Up [Azure Portal](https://portal.azure.com/) and Log Into Customer Environment
1. Open Up Power Shell Command Line

1. First ensure that the `Microsoft.AzureActiveDirectory` provider is registered. Run in CLI
   1. Output of CLI Command: `registrationStat: Registered` (may take some time to run)
2. Define Vairbles for PowerShell Script, if we are doing a Deployment for both TEST and PROD, this will need to be revised and re-run 2 times

$AzureSubscriptionId="cfa79485-58c6-439c-95d6-5e2c86550350"
$AzureResourceGroup="TribalTest"
$B2CTenantName="TibalPlatformsTestB2C"
$Location="United States"
$CountryCode="US"
$ApiVersion = "?api-version=2021-04-01"



## Check If Tenant Exists

In [ ]:
if (!$AzureSubscriptionId) {
  Write-Host "Getting subscription ID from the current account..."
  $AzureSubscriptionId = $(az account show --query "id" -o tsv)
  Write-Host $AzureSubscriptionId
}

$resourceId = 
"/subscriptions/$AzureSubscriptionId/resourceGroups/$AzureResourceGroup/providers/Microsoft.AzureActiveDirectory/b2cDirectories/$B2CTenantName.onmicrosoft.com?$ApiVersion"

# Check if tenant already exists
Write-Host "Checking if tenant '$B2CTenantName' already exists..."
az resource show --id $resourceId | Out-Null
if($LastExitCode -eq 0) # No error means, the resource exists
{
  Write-Warning "Tenant '$B2CTenantName' already exists. Not attempting to recreate it."
  return

## Confirm tennant does not exist, run the following Script

In [ ]:
$AzureSubscriptionId = "cfa79485-58c6-439c-95d6-5e2c86550350"
$AzureResourceGroup = "TribalDev"
$B2CTenantName = "TribalPlatformsDevB2C"
$Location = "United States"
$CountryCode = "US"
$ApiVersion = "?api-version=2021-04-01"

# Define the $reqBody structure as per your request
$reqBody = @{
    location = $Location
    sku = ""
}

# Convert the PowerShell object to JSON and flatten it
$jsonReqBody = $reqBody | ConvertTo-Json -Depth 10
$jsonReqBody = $jsonReqBody -replace "`n", "" -replace "`r", "" -replace " ", ""

# Output the flattened JSON for verification
Write-Output $jsonReqBody
Write-Output $Url

Write-Host "Creating B2C tenant $B2CTenantName..."
# https://docs.microsoft.com/en-us/rest/api/activedirectory/b2c-tenants/create
$Url = "https://management.azure.com/subscriptions/$AzureSubscriptionId/resourceGroups/$AzureResourceGroup/providers/Microsoft.AzureActiveDirectory/b2cDirectories/$B2CTenantName$ApiVersion"

# Execute the az rest command
az rest --method put `
    --url $Url `
    --body $jsonReqBody `
    # --headers "Content-Type=application/json"

if($LastExitCode -ne 0)
{
    throw "Error on creating new B2C tenant!"
}

Write-Host "*** B2C Tenant creation started. It can take a moment to complete."

do
{
  Write-Host "Waiting for 30 seconds for B2C tenant creation..."
  Start-Sleep -Seconds 30

  az resource show --id $resourceId
}
while($LastExitCode -ne 0)

In [ ]:
function Invoke-TenantInit {
  param (
    [string] $B2CTenantName
  )

  $B2CTenantId = "$($B2CTenantName).onmicrosoft.com"

  # Get access token for the B2C tenant with audience "management.core.windows.net".
  $managementAccessToken = $(az account get-access-token --tenant "$B2CTenantId" --query accessToken -o tsv)

  # Invoke tenant initialization which happens through the portal automatically.
  Write-Host "Invoking tenant initialization..."
  Invoke-WebRequest -Uri "https://main.b2cadmin.ext.azure.com/api/tenants/GetAndInitializeTenantPolicy?tenantId=$($B2CTenantId)&skipInitialization=false" `
    -Method "GET" `
    -Headers @{
      "Authorization" = "Bearer $($managementAccessToken)"
    }
}

# Full Script

In [ ]:
# Variables
$AzureSubscriptionId = "cfa79485-58c6-439c-95d6-5e2c86550350"
$AzureResourceGroup = "TribalTest"
$B2CTenantName = "TribalPlatformsTestB2C"
$Location = "United States"
$CountryCode = "US"
$ApiVersion = "2021-04-01"

# Set the subscription
az account set --subscription $AzureSubscriptionId

# Prepare the request body
$requestBody = @{
  location = $Location
  properties = @{
    displayName = $B2CTenantName
    countryCode = $CountryCode
  }
} | ConvertTo-Json -Depth 3

# Send the PUT request to create the B2C tenant and the corresponding Azure resource
az rest --method put `
  --uri "https://management.azure.com/subscriptions/cfa79485-58c6-439c-95d6-5e2c86550350/resourceGroups/TribalDev/providers/Microsoft.AzureActiveDirectory/b2cDirectories/TribalPlatformsDevB2C?api-version=2021-04-01" `
  --body $requestBody

# Output the result
Write-Output "Azure AD B2C tenant creation request has been initiated."


In [12]:
const myObj = {
    name: "John",
    age: 30,
    myCars: {
      car1: "Ford",
      car2: "BMW",
      car3: "Fiat"
    }
  }
  
  let p1 = "myCars";
  let p2 = "car2";

  console.log(myObj[p1][p2]);

BMW